# Install & Load Packages

In [1]:
%pip install -U "sentence-transformers[train]" " transformers[torch]" accelerate datasets pandas matplotlib seaborn numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
# Import libraries
from datasets import load_dataset, Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)
from sentence_transformers.losses import CoSENTLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
import pandas as pd
import numpy as np

# Fine-tuning Model

In [3]:
# Define model
model = SentenceTransformer("firqaaa/indo-sentence-bert-base")

# Define loss function (CoSENTLoss | Cosine Sentence Loss -> Returning float similarity score)
loss = CoSENTLoss(model)

In [4]:
# Define datasets
train_dataset = load_dataset("csv", data_files="data/preprocessed-data.csv")
test_dataset = load_dataset("csv", data_files="data/dataset-question.csv")
eval_dataset = load_dataset("csv", data_files="data/dataset-eval.csv")
train_dataset, test_dataset

Generating train split: 247 examples [00:00, 3249.79 examples/s]


(DatasetDict({
     train: Dataset({
         features: ['pattern', 'tag'],
         num_rows: 247
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['pattern', 'tag'],
         num_rows: 259
     })
 }))

In [39]:
# Specify training args
args = SentenceTransformerTrainingArguments(
    output_dir="fine-tuned/sbert-fine-tuned-chatPMB",
    num_train_epochs=100,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
)

In [11]:
# Creating eval dataset
train_patterns = pd.read_csv('data/preprocessed-data.csv')['pattern']
test_patterns = pd.read_csv('data/dataset-question.csv')['pattern']
test_patterns = test_patterns.iloc[:-12]
print(train_patterns.shape, test_patterns.shape)

(247,) (247,)


In [12]:
embed_train = model.encode(train_patterns)
embed_test = model.encode(test_patterns)

In [71]:
scores = []
sentences_1 = []
sentences_2 = []

for i in range(len(test_patterns)):
    data = model.similarity(embed_train[int(i)], embed_test[int(i)])
    data = float(data[0][0])
    s1 = train_patterns[int(i)]
    s2 = test_patterns[int(i)]
    sentences_1.append(s1)
    sentences_2.append(s2)
    scores.append(data)

df = pd.DataFrame({
    "sentence_1": sentences_1,
    "sentence_2": sentences_2,
    "label": scores,
})
df.to_csv('data/fine-tuned-dataset.csv')
df.tail()

,sentence_1,sentence_2,label
242,uin sunan gunung djati bandung punya beasiswa,Bisakah Anda memberitahu saya apakah di UIN Su...,0.845474
243,info soal beasiswa uin sunan gunung djati bandung,Dapatkah Anda memberikan informasi tentang ada...,0.864547
244,beasiswa uin sunan gunung djati bandung info l...,Mohon penjelasan apakah ada program beasiswa d...,0.815647
245,minta info dong uin sunan gunung djati bandung...,Saya ingin menanyakan apakah di UIN Sunan Gunu...,0.862830
246,beasiswa uin sunan gunung djati bandung gak sih,Mohon konfirmasi apakah UIN Sunan Gunung Djati...,0.779388


In [47]:
# Create evaluator & evaluate the base model
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=df["sentence_1"],
    sentences2=df["sentence_2"],
    scores=df["label"],
    main_similarity=SimilarityFunction.COSINE,
    show_progress_bar=True,
    precision="float32",
    name="train-evaluator",
)
dev_evaluator(model)

















Batches: 100%|██████████| 16/16 [00:19<00:00,  1.22s/it]

















Batches: 100%|██████████| 16/16 [00:19<00:00,  1.21s/it]


{'train-evaluator_pearson_cosine': 0.9999999999998636,
 'train-evaluator_spearman_cosine': 0.999995022854554,
 'train-evaluator_pearson_manhattan': 0.9370126899993185,
 'train-evaluator_spearman_manhattan': 0.9988644150622566,
 'train-evaluator_pearson_euclidean': 0.9386299766186968,
 'train-evaluator_spearman_euclidean': 0.9999920365732318,
 'train-evaluator_pearson_dot': 0.9999999999999291,
 'train-evaluator_spearman_dot': 0.9999962173872572,
 'train-evaluator_pearson_max': 0.9999999999999291,
 'train-evaluator_spearman_max': 0.9999962173872572}

In [48]:
training_data = Dataset.from_dict(df)
training_data, train_dataset

(Dataset({
     features: ['sentence_1', 'sentence_2', 'label'],
     num_rows: 247
 }),
 DatasetDict({
     train: Dataset({
         features: ['pattern', 'tag'],
         num_rows: 247
     })
 }))

In [49]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=training_data,
    # eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

  9%|▉         | 100/1100 [20:12<2:44:34,  9.87s/it]

{'loss': 4.1953, 'grad_norm': 155.7838592529297, 'learning_rate': 1.8181818181818182e-05, 'epoch': 9.09}


  9%|▉         | 100/1100 [20:45<2:44:34,  9.87s/it]

{'eval_train-evaluator_pearson_cosine': 0.8093898640561992, 'eval_train-evaluator_spearman_cosine': 0.824334797888826, 'eval_train-evaluator_pearson_manhattan': 0.7798400142510288, 'eval_train-evaluator_spearman_manhattan': 0.8241380285194678, 'eval_train-evaluator_pearson_euclidean': 0.7826685689888642, 'eval_train-evaluator_spearman_euclidean': 0.8243323361809636, 'eval_train-evaluator_pearson_dot': 0.8093898617977469, 'eval_train-evaluator_spearman_dot': 0.8243353547107994, 'eval_train-evaluator_pearson_max': 0.8093898640561992, 'eval_train-evaluator_spearman_max': 0.8243353547107994, 'eval_runtime': 32.1151, 'eval_samples_per_second': 0.0, 'eval_steps_per_second': 0.0, 'epoch': 9.09}


 18%|█▊        | 200/1100 [42:25<2:37:33, 10.50s/it]

...

{'eval_train-evaluator_pearson_cosine': 0.6899927880800748, 'eval_train-evaluator_spearman_cosine': 0.77506730611496, 'eval_train-evaluator_pearson_manhattan': 0.6575149776113081, 'eval_train-evaluator_spearman_manhattan': 0.7749200571311735, 'eval_train-evaluator_pearson_euclidean': 0.6583362837014505, 'eval_train-evaluator_spearman_euclidean': 0.7750649915344203, 'eval_train-evaluator_pearson_dot': 0.6899927847975237, 'eval_train-evaluator_spearman_dot': 0.7750636027960498, 'eval_train-evaluator_pearson_max': 0.6899927880800748, 'eval_train-evaluator_spearman_max': 0.77506730611496, 'eval_runtime': 39.2342, 'eval_samples_per_second': 0.0, 'eval_steps_per_second': 0.0, 'epoch': 100.0}


100%|██████████| 1100/1100 [3:38:49<00:00, 10.31s/it]

{'train_runtime': 13129.7054, 'train_samples_per_second': 1.881, 'train_steps_per_second': 0.084, 'train_loss': 2.947750868363814, 'epoch': 100.0}


100%|██████████| 1100/1100 [3:38:50<00:00, 11.94s/it]


TrainOutput(global_step=1100, training_loss=2.947750868363814, metrics={'train_runtime': 13129.7054, 'train_samples_per_second': 1.881, 'train_steps_per_second': 0.084, 'total_flos': 0.0, 'train_loss': 2.947750868363814, 'epoch': 100.0})

In [61]:
model.save_pretrained("fine-tuned/models/chatPMB-pretrained-2")

# Get Tags by index

In [84]:
labels = pd.read_csv('data/dataset-question.csv')
labels = labels.iloc[:-12]
labels = labels['tag']
labels

0         gr_hi
1         gr_ha
2         gr_pa
3         gr_si
4         gr_so
         ...   
242    beasiswa
243    beasiswa
244    beasiswa
245    beasiswa
246    beasiswa
Name: tag, Length: 247, dtype: object

# Usage of Pretrained Model

In [63]:
model = SentenceTransformer("fine-tuned/models/chatPMB-pretrained-2")

In [74]:
sentence = model.encode("Saya sedang mencari informasi seputar biaya UKT di UIN Sunan Gunung Djati Bandung")
to_test = model.encode(df['sentence_2'])

In [86]:
result = np.array(model.similarity(sentence, to_test))
index = np.argmax(result)
detected_label = labels[index]
index, detected_label, result

(224,
 'ukt',
 array([[-0.08102501, -0.09274451, -0.0194903 , -0.09801254, -0.07346375,
          0.01122001, -0.04107093,  0.02444862,  0.00458639,  0.16658112,
          0.376706  ,  0.2224128 ,  0.39654344,  0.12525932,  0.1698928 ,
          0.33939266,  0.18692902,  0.42380178,  0.15407269, -0.02946443,
          0.0310082 , -0.04368572,  0.10426041,  0.04318966,  0.11615118,
          0.08738878,  0.00334094,  0.05256866,  0.0368742 ,  0.0036604 ,
         -0.08304228, -0.1332272 , -0.13874346,  0.08654732,  0.2074686 ,
          0.1313885 ,  0.44935143,  0.14125264,  0.2567071 ,  0.4249447 ,
          0.18028516,  0.20853628,  0.37663478,  0.05177975,  0.14704998,
          0.17423037,  0.21683279,  0.17834032,  0.02997787,  0.05023821,
          0.12785752,  0.0825028 ,  0.04757181,  0.05177975,  0.14704998,
          0.17423037,  0.21683279,  0.17834032,  0.02997787,  0.05023821,
          0.12785752,  0.0825028 ,  0.04757181,  0.12470879,  0.08080865,
          0.15623328,  0